# 10: GNSS Environment Models

This workshop dives into the physical models used for realistic GNSS signal
simulation: orbital mechanics, ionospheric/tropospheric delays, multipath,
and antenna patterns.

## Learning Objectives
- Understand Keplerian orbit propagation and GPS satellite geometry
- Compute ionospheric delay using the Klobuchar model
- Calculate tropospheric delay with Saastamoinen model
- Model multipath environments (open sky, urban canyon, indoor)
- Analyze antenna gain patterns and their effect on C/N0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Constants
GM = 3.986004418e14      # Earth gravitational parameter (m^3/s^2)
OMEGA_E = 7.2921150e-5   # Earth rotation rate (rad/s)
C = 299792458.0          # Speed of light (m/s)
R_EARTH = 6378137.0      # WGS-84 semi-major axis (m)
F_L1 = 1575.42e6         # GPS L1 frequency (Hz)

## Part 1: Keplerian Orbits

GPS satellites orbit at ~20,200 km altitude with a period of ~11h 58m
(half a sidereal day). The orbit is defined by 6 Keplerian elements:

| Element | Symbol | GPS Value |
|---------|--------|----------|
| Semi-major axis | a | 26,559.7 km |
| Eccentricity | e | ~0 (circular) |
| Inclination | i | 55 deg |
| RAAN | Omega | 60 deg spacing |
| Arg of perigee | omega | 0 |
| Mean anomaly | M0 | slot-dependent |

The 24+ GPS satellites are arranged in 6 orbital planes.

In [ ]:
# GPS orbital parameters
a_gps = 26559700.0  # meters
i_gps = np.radians(55.0)

# Mean motion and period
n = np.sqrt(GM / a_gps**3)
T = 2 * np.pi / n

print(f'GPS Orbital Parameters:')
print(f'  Semi-major axis: {a_gps/1000:.1f} km')
print(f'  Altitude: {(a_gps - R_EARTH)/1000:.1f} km')
print(f'  Mean motion: {n*1e6:.4f} x10^-6 rad/s')
print(f'  Period: {T:.0f} s = {T/3600:.2f} hours')
print(f'  Orbital velocity: {2*np.pi*a_gps/T:.0f} m/s = {2*np.pi*a_gps/T/1000:.2f} km/s')

In [ ]:
def kepler_to_eci(a, e, i, raan, omega, M, t):
    """Convert Keplerian elements to ECI coordinates."""
    # Solve Kepler's equation: M = E - e*sin(E)
    E = M.copy()
    for _ in range(20):
        E = E - (E - e*np.sin(E) - M) / (1 - e*np.cos(E))
    
    # True anomaly
    nu = 2 * np.arctan2(np.sqrt(1+e)*np.sin(E/2), np.sqrt(1-e)*np.cos(E/2))
    
    # Radius
    r = a * (1 - e*np.cos(E))
    
    # Position in orbital plane
    x_orb = r * np.cos(nu)
    y_orb = r * np.sin(nu)
    
    # Rotation to ECI
    cos_o = np.cos(omega); sin_o = np.sin(omega)
    cos_r = np.cos(raan); sin_r = np.sin(raan)
    cos_i = np.cos(i); sin_i = np.sin(i)
    
    x = (cos_r*cos_o - sin_r*sin_o*cos_i)*x_orb + (-cos_r*sin_o - sin_r*cos_o*cos_i)*y_orb
    y = (sin_r*cos_o + cos_r*sin_o*cos_i)*x_orb + (-sin_r*sin_o + cos_r*cos_o*cos_i)*y_orb
    z = (sin_o*sin_i)*x_orb + (cos_o*sin_i)*y_orb
    
    return x, y, z

# Plot full GPS constellation (6 planes x 4 satellites)
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

colors = plt.cm.tab10(np.linspace(0, 1, 6))
t_orbit = np.linspace(0, T, 200)

for plane in range(6):
    raan = np.radians(60 * plane)
    # Plot orbital path
    M = np.sqrt(GM/a_gps**3) * t_orbit
    x, y, z = kepler_to_eci(a_gps, 0, i_gps, raan, 0, M, t_orbit)
    ax.plot(x/1e6, y/1e6, z/1e6, color=colors[plane], alpha=0.3, linewidth=0.5)
    
    # Plot satellite positions
    for slot in range(4):
        M0 = np.radians(60 * slot + 15 * plane)  # offset between planes
        xs, ys, zs = kepler_to_eci(a_gps, 0, i_gps, raan, 0, np.array([M0]), 0)
        ax.scatter(xs/1e6, ys/1e6, zs/1e6, color=colors[plane], s=50,
                  label=f'Plane {plane+1}' if slot == 0 else '')

# Earth sphere
u = np.linspace(0, 2*np.pi, 30)
v = np.linspace(0, np.pi, 20)
xe = R_EARTH/1e6 * np.outer(np.cos(u), np.sin(v))
ye = R_EARTH/1e6 * np.outer(np.sin(u), np.sin(v))
ze = R_EARTH/1e6 * np.outer(np.ones_like(u), np.cos(v))
ax.plot_surface(xe, ye, ze, alpha=0.2, color='blue')

ax.set_xlabel('X (1000 km)')
ax.set_ylabel('Y (1000 km)')
ax.set_zlabel('Z (1000 km)')
ax.set_title('GPS Constellation (6 planes, 24 satellites)')
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()

## Part 2: Ionospheric Delay (Klobuchar Model)

The ionosphere (60-1000 km altitude) contains free electrons that delay GNSS signals.
Key properties:

- **Group delay scales as 1/f^2** - lower frequencies are delayed more
- **Varies with time of day** - peak around 14:00 local time
- **Varies with latitude** - maximum near equator
- **Depends on elevation** - low elevation = longer path through ionosphere

The Klobuchar model uses 8 broadcast coefficients (alpha/beta) to estimate delay.

In [ ]:
def klobuchar_delay(elevation_rad, azimuth_rad, lat_rad, lon_rad, gps_time_s,
                    alpha=[0.1118e-7, 0.7451e-8, -0.5961e-7, -0.1192e-6],
                    beta=[0.1167e6, -0.4267e5, -0.2621e6, 0.1311e6]):
    """Klobuchar ionospheric delay model."""
    el = elevation_rad / np.pi  # semicircles
    az = azimuth_rad / np.pi
    lat = lat_rad / np.pi
    lon = lon_rad / np.pi
    
    psi = 0.0137 / (el + 0.11) - 0.022
    lat_ipp = np.clip(lat + psi * np.cos(az * np.pi), -0.416, 0.416)
    lon_ipp = lon + psi * np.sin(az * np.pi) / np.cos(lat_ipp * np.pi)
    lat_mag = lat_ipp + 0.064 * np.cos((lon_ipp - 1.617) * np.pi)
    
    t = (43200 * lon_ipp + gps_time_s) % 86400
    F = 1.0 + 16.0 * (0.53 - el)**3
    
    amp = max(0, sum(a * lat_mag**i for i, a in enumerate(alpha)))
    per = max(72000, sum(b * lat_mag**i for i, b in enumerate(beta)))
    
    x = 2 * np.pi * (t - 50400) / per
    if abs(x) < 1.57:
        delay = F * (5e-9 + amp * (1 - x**2/2 + x**4/24))
    else:
        delay = F * 5e-9
    
    return delay  # seconds

# Plot ionospheric delay vs elevation and time
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Vs elevation
elevations = np.linspace(5, 90, 50)
delays_m = [klobuchar_delay(np.radians(el), 0, np.radians(40), np.radians(-75), 43200) * C
            for el in elevations]
axes[0].plot(elevations, delays_m)
axes[0].set_xlabel('Elevation (degrees)')
axes[0].set_ylabel('Delay (meters)')
axes[0].set_title('Ionospheric Delay vs Elevation (noon, 40N)')

# Vs time of day
times = np.linspace(0, 86400, 200)
delays_tod = [klobuchar_delay(np.radians(45), 0, np.radians(40), np.radians(-75), t) * C
              for t in times]
axes[1].plot(times/3600, delays_tod)
axes[1].set_xlabel('GPS Time (hours)')
axes[1].set_ylabel('Delay (meters)')
axes[1].set_title('Ionospheric Delay vs Time of Day (45 deg elev, 40N)')

plt.tight_layout()
plt.show()

print(f'Zenith delay at noon: {delays_m[-1]:.1f} m ({delays_m[-1]/C*1e9:.1f} ns)')
print(f'Low elevation (10 deg) delay: {delays_m[2]:.1f} m')

## Part 3: Tropospheric Delay (Saastamoinen Model)

The troposphere (0-50 km) delays signals due to the neutral atmosphere.
Unlike the ionosphere, tropospheric delay is:

- **Non-dispersive** - same delay at all frequencies
- **Two components**: Hydrostatic (dry, ~90%) and Wet (~10%)
- **Elevation-dependent** via mapping function (~1/sin(el))
- **Zenith delay ~2.3m** at sea level standard atmosphere

In [ ]:
def saastamoinen_delay(elevation_rad, height_m=0, temp_k=288.15, pressure_hpa=1013.25, humidity=0.5):
    """Saastamoinen tropospheric delay model."""
    # Hydrostatic zenith delay
    zhd = 0.002277 * pressure_hpa / (1 - 0.00266*np.cos(2*np.radians(height_m)) - 0.00028*height_m/1000)
    
    # Water vapor pressure (Buck equation)
    t_c = temp_k - 273.15
    es = 6.1121 * np.exp((18.678 - t_c/234.5) * t_c / (257.14 + t_c))
    e = humidity * es
    
    # Wet zenith delay
    zwd = 0.002277 * (1255/temp_k + 0.05) * e
    
    # Mapping function
    el = max(elevation_rad, 0.05)
    mf = 1.0 / (np.sin(el) + 0.00143 / np.tan(0.0455 + np.sin(el)))
    
    return (zhd + zwd) * mf  # meters

# Compare elevation dependence
elevations = np.linspace(5, 90, 100)
tropo_delay = [saastamoinen_delay(np.radians(el)) for el in elevations]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(elevations, tropo_delay)
axes[0].set_xlabel('Elevation (degrees)')
axes[0].set_ylabel('Delay (meters)')
axes[0].set_title('Tropospheric Delay vs Elevation')
axes[0].axhline(y=saastamoinen_delay(np.radians(90)), color='r', linestyle='--', label='Zenith')
axes[0].legend()

# Compare with altitude
altitudes = np.linspace(0, 5000, 50)
zenith_delays = []
for h in altitudes:
    lapse = 0.0065
    t = 288.15 - lapse * h
    p = 1013.25 * (1 - lapse*h/288.15)**5.2561
    zenith_delays.append(saastamoinen_delay(np.radians(90), h, t, p))

axes[1].plot(altitudes/1000, zenith_delays)
axes[1].set_xlabel('Altitude (km)')
axes[1].set_ylabel('Zenith Delay (meters)')
axes[1].set_title('Zenith Tropospheric Delay vs Altitude')

plt.tight_layout()
plt.show()

print(f'Sea level zenith delay: {saastamoinen_delay(np.radians(90)):.3f} m')
print(f'Low elevation (5 deg): {saastamoinen_delay(np.radians(5)):.1f} m')

## Part 4: Multipath Environments

GNSS multipath occurs when signals reflect off nearby surfaces. The effect depends on:

| Environment | Direct Path | Reflections | Typical Error |
|-------------|-------------|-------------|---------------|
| Open Sky | Strong | Minimal | <1m |
| Suburban | Strong | Moderate | 1-5m |
| Urban Canyon | Attenuated | Severe | 5-50m |
| Indoor | Weak | Dominant | 10-100m |

In [ ]:
# Define multipath tap profiles (delay_ns, power_dB)
environments = {
    'Open Sky': [(0, 0)],
    'Suburban': [(0, 0), (50, -6), (120, -12)],
    'Urban Canyon': [(0, 0), (30, -3), (80, -5), (200, -8), (500, -14)],
    'Indoor': [(0, -3), (20, -2), (50, -4), (100, -6), (200, -10), (400, -15)],
}

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, (name, taps) in zip(axes.flat, environments.items()):
    delays = [t[0] for t in taps]
    powers = [t[1] for t in taps]
    
    ax.stem(delays, powers, linefmt='b-', markerfmt='bo', basefmt='r-')
    ax.set_xlabel('Delay (ns)')
    ax.set_ylabel('Power (dB)')
    ax.set_title(f'{name} - {len(taps)} tap(s)')
    ax.set_ylim(-20, 2)
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## Part 5: Antenna Gain Patterns

The receiver antenna pattern determines how much signal is captured from each
satellite direction. GNSS antennas are designed to favor high-elevation satellites
and reject low-elevation (multipath-prone) signals.

In [ ]:
def patch_gain(elevation_deg, peak_dbi=5.0, beamwidth_deg=150.0):
    """Microstrip patch antenna gain."""
    if elevation_deg < -5:
        return -30
    theta = np.radians(90 - elevation_deg)
    half_bw = np.radians(beamwidth_deg / 2)
    n = np.log10(3) / np.log10(1/np.cos(half_bw))
    gain_lin = abs(np.cos(theta))**n
    gain_db = 10*np.log10(max(gain_lin, 1e-6))
    return peak_dbi + gain_db

def choke_ring_gain(elevation_deg, peak_dbi=7.0):
    """Choke ring geodetic antenna."""
    if elevation_deg < 0:
        return -40
    theta = np.radians(90 - elevation_deg)
    gain_lin = abs(np.cos(theta))**1.5
    gain_db = 10*np.log10(max(gain_lin, 1e-6))
    return peak_dbi + gain_db

# Polar plot of antenna patterns
fig, ax = plt.subplots(1, 1, figsize=(8, 8), subplot_kw={'projection': 'polar'})

elevations = np.linspace(-10, 90, 200)
theta = np.radians(90 - elevations)  # 0=zenith

patterns = {
    'Isotropic': [0] * len(elevations),
    'Patch (5 dBi)': [patch_gain(el) for el in elevations],
    'Choke Ring (7 dBi)': [choke_ring_gain(el) for el in elevations],
}

for name, gains in patterns.items():
    ax.plot(theta, gains, label=name, linewidth=2)

ax.set_theta_zero_location('N')  # zenith at top
ax.set_theta_direction(-1)
ax.set_rlabel_position(45)
ax.set_title('GNSS Antenna Gain Patterns', pad=20)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

## Part 6: Complete Link Budget

Putting it all together: the received C/N0 for each satellite:

$$C/N_0 = EIRP - FSPL + G_{ant} - kT$$

Where:
- EIRP: ~14.3 dBW (GPS L1 transmit power)
- FSPL: ~182 dB (at 20,200 km, L1 frequency)
- G_ant: antenna gain at satellite elevation
- kT: -228.6 dBW/K/Hz (Boltzmann constant x temperature)

In [ ]:
# Link budget calculation
EIRP_DBW = 14.3        # GPS L1 transmit EIRP
KT_DBWHZ = -228.6      # kT at 290K
NF_DB = 2.0            # Receiver noise figure

def link_budget(elevation_deg, antenna='patch'):
    """Calculate C/N0 for a GPS satellite at given elevation."""
    # Range (approximate, from elevation)
    r_sat = 26559700  # GPS orbit radius
    # Slant range from elevation
    el_rad = np.radians(elevation_deg)
    range_m = -R_EARTH*np.sin(el_rad) + np.sqrt((R_EARTH*np.sin(el_rad))**2 + r_sat**2 - R_EARTH**2)
    
    # Free space path loss
    fspl = 20*np.log10(4*np.pi*range_m*F_L1/C)
    
    # Antenna gain
    if antenna == 'patch':
        g_ant = patch_gain(elevation_deg)
    else:
        g_ant = choke_ring_gain(elevation_deg)
    
    # C/N0
    cn0 = EIRP_DBW - fspl + g_ant - KT_DBWHZ - NF_DB
    
    return cn0, fspl, g_ant, range_m

# Plot C/N0 vs elevation for different antennas
elevations = np.linspace(5, 90, 50)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ant_name in ['patch', 'choke_ring']:
    cn0s = [link_budget(el, ant_name)[0] for el in elevations]
    axes[0].plot(elevations, cn0s, label=f'{ant_name.replace("_", " ").title()} Antenna')

axes[0].axhline(y=35, color='r', linestyle='--', label='Tracking threshold')
axes[0].axhline(y=30, color='orange', linestyle='--', label='Acquisition threshold')
axes[0].set_xlabel('Elevation (degrees)')
axes[0].set_ylabel('C/N0 (dB-Hz)')
axes[0].set_title('C/N0 vs Satellite Elevation')
axes[0].legend()
axes[0].set_ylim(20, 55)

# Budget breakdown at 45 deg
cn0, fspl, g_ant, range_m = link_budget(45)
items = ['EIRP', 'FSPL', 'Ant Gain', 'kT+NF', 'C/N0']
values = [EIRP_DBW, -fspl, g_ant, KT_DBWHZ + NF_DB, cn0]
colors = ['green', 'red', 'green', 'red', 'blue']

axes[1].barh(items, values, color=colors)
axes[1].set_xlabel('dBW or dB-Hz')
axes[1].set_title('Link Budget Breakdown (45 deg elevation)')
for i, v in enumerate(values):
    axes[1].text(v + 1 if v > 0 else v - 8, i, f'{v:.1f}', va='center')

plt.tight_layout()
plt.show()

print(f'\nAt 45 deg elevation:')
print(f'  Range: {range_m/1000:.0f} km')
print(f'  FSPL: {fspl:.1f} dB')
print(f'  Antenna gain: {g_ant:.1f} dBi')
print(f'  C/N0: {cn0:.1f} dB-Hz')

## Exercises

1. **Dual-frequency ionosphere correction**: Calculate the ionospheric delay
   at both L1 (1575.42 MHz) and L5 (1176.45 MHz). Use the L1-L5 combination
   to estimate and remove the ionospheric error.

2. **Elevation mask analysis**: Plot the number of visible GPS satellites
   (above mask angle) as a function of mask angle from 0 to 30 degrees.

3. **DOP calculation**: Given 4+ visible satellites, compute the Geometric
   Dilution of Precision (GDOP) from the satellite geometry.

4. **Seasonal troposphere**: Investigate how temperature and humidity
   variations (summer vs winter) affect tropospheric delay.

In [ ]:
# Your code here
